# Satellite Image Segmentation of Nepal
**Prepared By:** Ajeeb Rimal | M. Tech. AI | Kathmandu University

## Directory definitions and setup
We have defined the directory paths for the dataset, image, and mask files. We have also defined and created the output directory where the patches will be saved.Additionally we have defined the patch size, stride, and boundary for the patches. The boundary is defined as a list of four values: [min_x, min_y, max_x, max_y] which represent the minimum and maximum x and y coordinates of the area of interest, respectively. We have used the `os` library to create the output directory if it does not exist.

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os

In [ ]:
mission_list = ["Mission 1", "Mission 2", "Mission 3", "Mission 4", "Mission 5"]
dataset_dir = r"/Users/ajeebrimal/Documents/Masters Thesis/Datasets/Rupandehi Data"
annotations_dir = os.path.join(dataset_dir, "Annotations")

In [ ]:
output_dir = r"../output"
os.makedirs(output_dir, exist_ok=True)

rasterized_dir = os.path.join(output_dir, "rasterized_outputs")
os.makedirs(rasterized_dir, exist_ok=True)

In [ ]:
image_dir = os.path.join(dataset_dir, "TIF Files")
mask_dirs = [os.path.join(annotations_dir, mask_dir) for mask_dir in mission_list]
mask_paths = [os.path.join(mask_dir, f"{mission}.shp") for mask_dir, mission in zip(mask_dirs, mission_list)]
image_paths = [os.path.join(image_dir, f"{mission}.tif") for mission in mission_list]
mask_paths_rasterized = [os.path.join(rasterized_dir, f"{mission}_rasterized.tif") for mission in mission_list]

In [ ]:
patch_size = 512
stride = 256

patch_output_dir = os.path.join(output_dir, f"{patch_size}x{patch_size}")
os.makedirs(patch_output_dir, exist_ok=True)

#### Log setup

In [ ]:
from sources.helpers.logger import LoggerHelper

# Ensure the log directory exists
logger_name = "Satellite Segmentation Nepal"
log_dir=r"../logs"

logger = LoggerHelper(logger_name=logger_name, log_dir=log_dir).logger
logger.info("Logger setup complete.")

### Utils Setup

In [ ]:
from sources.helpers.utils import UtilsHelper

utils = UtilsHelper()

## Dataset preparation

### Rasterize the mask
Rasterize the mask to the same projection and pixel resolution as the reference image. We have used the `gdal` library to rasterize the mask. The `gdal` library is a translator library for raster and vector geospatial data formats. It also includes a variety of useful command-line utilities for data translation and processing.

In [ ]:
for image_file_path, mask_shape_file_path in zip(image_paths, mask_paths):
    utils.rasterize_masks(image_file_path, mask_shape_file_path,rasterized_dir)

### Create image and mask patches

To prepare the satellite images and masks for segmentation, we first need to convert them into smaller patches. Here's how we can do it:

1. Use the `create_patches()` function from the `utils` module.
2. Pass the following parameters to the function:
    - The file path of the satellite image in `.tif` format.
    - The file path of the mask in `.shp` format.
    - The output directory where the created patches will be saved.
    - The patch size (we use the same value for height and width).
    - The stride.
    - A boundary that defines the area of interest.
3. The function will iterate over the satellite image in patches of the specified size and stride.
4. For each patch, the function will check if it intersects with any mask geometries (which are shapes such as polygons, lines, points, etc. that represent the features belonging to certain classes).
5. If the patch intersects with any mask geometries, the function will create a patch mask by rasterizing the intersecting mask geometries. Otherwise, it will discard the patch.
6. The function will save the patch and its corresponding patch mask to the output directory.

In [ ]:
min_mask_coverage = 0.1 # Percentage of minimum mask coverage in the patch

# Iterate over image and mask paths
for image_path, mask_path in zip(image_paths, mask_paths_rasterized):
    utils.create_patches_categorical(
        image_path=image_path,
        mask_path=mask_path,
        output_dir=patch_output_dir,
        patch_size=patch_size,
        stride=stride,
        min_mask_coverage=min_mask_coverage
    )

### Split the dataset into training and validation sets
This process organizes satellite image data and corresponding masks into `train`, `val`, and `test` directories while maintaining alignment between images and masks.


In [ ]:
utils.split_dataset(patch_output_dir)

### Merge the split dataset back into a unified dataset

In [ ]:
utils.merge_split_dataset(patch_output_dir)

### Visualize a random image patch and its mask
We can visualize a random patch and its mask using the `rasterio` and `matplotlib` libraries. The `rasterio` library is a Python package that provides a fast and direct way to work with raster data. We can use the `rasterio.open()` function to open the image and mask files. We can then use the `read()` function to read the image and mask data. We can then use the `imshow()` function from the `matplotlib.pyplot` module to display the image and the mask.

In [ ]:
utils.visualize_random_patch(patch_output_dir)

### Dataset and data generator

We define the dataset and data generator that will be used for training the model. To define the dataset and data generator, we can use the `NepalDataset` and `NepalDataGenerator` classes from the `dataset` module. This code takes a dataset and generates batches of data for training a deep learning model.

In [ ]:
from sources.helpers.generator import NepalDataset, NepalDataGenerator
from torchvision import transforms

# Define dataset and data generator parameters
dataset_split_dir = os.path.join(patch_output_dir, "dataset_split")
batch_size = 5
shuffle = True
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create datasets for train, val, and test splits
train_dataset = NepalDataset(dataset_split_dir, split="train", transform=transform)
val_dataset = NepalDataset(dataset_split_dir, split="val", transform=transform)
test_dataset = NepalDataset(dataset_split_dir, split="test", transform=transform)

# Print dataset sizes
logger.debug(f"Train dataset size: {len(train_dataset)}")
logger.debug(f"Validation dataset size: {len(val_dataset)}")
logger.debug(f"Test dataset size: {len(test_dataset)}")

# Create data generators for each dataset
train_data_generator = NepalDataGenerator(train_dataset, batch_size=batch_size, shuffle=shuffle)
val_data_generator = NepalDataGenerator(val_dataset, batch_size=batch_size, shuffle=shuffle)
test_data_generator = NepalDataGenerator(test_dataset, batch_size=batch_size, shuffle=False)  # Do not shuffle test data

# Print data generator sizes
logger.debug(f"Train data generator size: {len(train_data_generator)}")
logger.debug(f"Validation data generator size: {len(val_data_generator)}")
logger.debug(f"Test data generator size: {len(test_data_generator)}")

### Visualize data generator output image and mask patches

This code visualizes the patches and patch masks that were created using the data generator. To visualize the patches and patch masks, we can use the `visualize()` function from `utils` module.

In [ ]:
no_of_images_to_show = 3  # Number of images to show from the batch
# Select the data_generator to visualize (train/validation/test)
selected_data_generator = train_data_generator  # Change to val_data_generator or test_data_generator as needed
# Get the batch of images and masks
utils.visualize_data_generator_output(
    selected_data_generator=selected_data_generator,
    no_of_images_to_show=no_of_images_to_show,
    batch_size=batch_size
)

## Training

### Model Definition


In [ ]:
from sources.helpers.train import TrainingHelper

# model_name= "deeplabv3_resnet50"
# model_name = "deeplabv3_mobilenet_v3_large"
model_name = "deeplabv3_mobilenet_v3_large"
num_epochs = 100
num_classes = 2
loss_function = "cross_entropy"
optimizer = "adam"

training = TrainingHelper(
    model_name = model_name,
    num_classes = num_classes,
    num_epochs = num_epochs,
    loss_function = loss_function,
    optimizer = optimizer,
    train_data_generator = train_data_generator,
    val_data_generator = val_data_generator,
)

In [ ]:
import mlflow

try:
    trained_model = training.train_model_with_mlflow(output_dir=output_dir)
except Exception as e:
    logger.error(f"Error occurred during training: {e}")
finally:
    mlflow.end_run()